# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query #1:

For this query the features artist, song_title and song_length are needed and the filter uses the features session_id and item_in_session. 

**Concept for Table**:
- The table will be named **song_library**.
- The primary key will contain the features session_id and item_in_session so the table can be filteres by these features later on. 
- Columns and datatype of the table will be session_id INT, item_in_session INT, artist TEXT, song_title TEXT and song_length FLOAT. 

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_create_table_1 = "CREATE TABLE IF NOT EXISTS song_library"
query_create_table_1 = query_create_table_1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT, song_length FLOAT, PRIMARY KEY (session_id, item_in_session))""")

try:
    session.execute(query_create_table_1)
except Exception as e:
    print(e)                 

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query_select_1 = """SELECT artist, song_title, song_length FROM song_library
                    WHERE session_id = %s
                    AND item_in_session = %s
                    """  
try:
    result_query_1 = session.execute(query_select_1, (338, 4))
except Exception as e:
    print(e)

df_1 = pd.DataFrame(list(result_query_1))

df_1.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query #2:

For this query the table should consist of the features artist_name, song_title, user_first_name and user_last_name. The filter for our query will need the features user_id and session_id. To sort the query by item_in_session the table also needs this as a feature. 

**Concept for Table**:
- The table will be named **artist_library**.
- The primary key will contain the features user_id, session_id and item_in_session. The features user_id and session_id will be used as a composite key. The feature item_in_session will be  used as the clustering key.
- Columns and dataype of the table are user_id INT, session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, user_first_name TEXT and user_last_name TEXT.

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_create_table_2 = "CREATE TABLE IF NOT EXISTS artist_library"
query_create_table_2 = query_create_table_2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                                    artist_name TEXT, song_title TEXT, user_first_name TEXT,
                                                    user_last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session))""")

try:
    session.execute(query_create_table_2)
except Exception as e:
    print(e)                          

In [12]:
## Insertion for query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist_name,\
                                             song_title, user_first_name, user_last_name)"
    
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## SELECT statement for query 2
query_select_2 = """SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_library
                    WHERE user_id = %s
                    AND session_id = %s
                    """  
try:
    result_query_2 = session.execute(query_select_2, (10, 182))
except Exception as e:
    print(e)

df_2 = pd.DataFrame(list(result_query_2))

df_2.head()

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query #3:

For this query the table will need the features user_first_name and user_last_name. The query will need the feature song_title as a filter. As user name and last names are often not uniwue in large datasets the feature user_id is added to the table.

**Concept for Table**:
- The table will be named **user_library**.
- The primary key will use song_title as a partition key and user_id as the clustering key.
- Columns and datatype of the table will be song_title TEXT, user_id INT, user_first_name TEXT and user_last_name TEXT

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_create_table_3 = "CREATE TABLE IF NOT EXISTS user_library"
query_create_table_3 = query_create_table_3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY (song_title, user_id))""")

try:
    session.execute(query_create_table_3)
except Exception as e:
    print(e)      

In [17]:
## Insertion for query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_library (song_title, user_id, user_first_name, user_last_name)"
    
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## SELECT statement for query 3
query_select_3 = """SELECT user_first_name, user_last_name FROM user_library
                    WHERE song_title = %s
                    """  
try:
    result_query_3 = session.execute(query_select_3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

df_3 = pd.DataFrame(list(result_query_3))

df_3.head()

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [23]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_library")
session.execute("DROP TABLE IF EXISTS artist_library")
session.execute("DROP TABLE IF EXISTS user_library")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()